In [ ]:
import urllib.request
import gzip

In [ ]:
localfile = 'pageviews-20190331-120000'

In [ ]:
url = 'https://dumps.wikimedia.org/other/pageviews/2019/2019-03/pageviews-20190331-120000.gz'

r = urllib.request.urlopen(url)
s_out = gzip.decompress(r.read())

with open(localfile, 'wb') as f:  
    f.write(s_out)

In [ ]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StringType, StructField, IntegerType

In [ ]:
spark = SparkSession.builder.getOrCreate()

In [ ]:
schema = StructType([StructField('code', StringType(), True),
                     StructField('title', StringType(), True),
                     StructField('nbrequest', IntegerType(), True),
                     StructField('total', IntegerType(), True)])

df = spark.read.csv(localfile, sep=' ', header=None, schema=schema)

In [ ]:
# Afficher les 10 elements les plus vus
df.groupBy('title').sum('nbrequest').orderBy('sum(nbrequest)', ascending = False).show(10)

In [ ]:
df.createOrReplaceTempView("wikipedia")

In [ ]:
# Equivalent en SQL
spark.sql("SELECT TITLE, SUM(NBREQUEST) FROM WIKIPEDIA GROUP BY TITLE ORDER BY SUM(NBREQUEST) DESC").show(10)

In [ ]:
# Afficher les 10 elements francais les plus vus
df.filter("code RLIKE '^fr'").groupBy('title').sum('nbrequest').orderBy('sum(nbrequest)', ascending = False).show(10)

In [ ]:
spark.sql("SELECT TITLE, SUM(NBREQUEST) FROM WIKIPEDIA WHERE CODE RLIKE '^fr' GROUP BY TITLE ORDER BY SUM(NBREQUEST) DESC").show(10)

In [ ]:
# Enlever les titres avec characteres speciaux (:)
spark.sql("SELECT TITLE, SUM(NBREQUEST) FROM WIKIPEDIA WHERE CODE RLIKE '^fr' AND NOT TITLE LIKE '%:%' GROUP BY TITLE ORDER BY SUM(NBREQUEST) DESC").show(25)